In [80]:
from pathlib import Path
import sys
import os
from pydantic import SecretStr

current_folder = Path.cwd()
src_root = current_folder.parent
project_root = src_root.parent
sys.path.insert(0, str(project_root))


from src.config import QDRANT_HOST, GEMINI_API_KEY, OPENAI_API_KEY

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue
from langchain_qdrant import QdrantVectorStore, RetrievalMode, FastEmbedSparse
from langchain_classic.tools.retriever import create_retriever_tool
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model
from langchain_core.tools import Tool
from src.retrieval.custom_tools import create_company_search_tool, create_all_companies_dense_search_tool
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from typing import List

print("Imports successful!")

Imports successful!


### Configuration

In [7]:
llm = ChatOpenAI(
    model="gpt-5-mini",
    api_key=SecretStr(OPENAI_API_KEY) if OPENAI_API_KEY else None,
)

In [11]:
COLLECTION_NAME = "dekkingen"
#EMBEDDING_MODEL_NAME = "models/embedding-001"
EMBEDDING_MODEL_NAME = "text-embedding-3-large" 
COLLECTION_NAME = "dekkingen"
EMBEDDING_DIMENSION = 3072  # embedding-001 produces 768-dimensional vectors
# Sparse embedding configuration for hybrid retrieval
SPARSE_MODEL_NAME = "Qdrant/bm25"  # BM25 sparse embeddings
DENSE_VECTOR_NAME = "dense"  # Name for dense vectors in Qdrant
SPARSE_VECTOR_NAME = "sparse"  # Name for sparse vectors in Qdrant

embeddings = OpenAIEmbeddings(
            model=EMBEDDING_MODEL_NAME,
            api_key=SecretStr(OPENAI_API_KEY) if OPENAI_API_KEY else None
        )

sparse_embeddings = FastEmbedSparse(model_name=SPARSE_MODEL_NAME)

### Initialize VectorStore

In [12]:
client = QdrantClient(host=QDRANT_HOST)
vector_store = QdrantVectorStore(
    client=client,
    collection_name="dekkingen",
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    vector_name="dense",
    sparse_vector_name="sparse",
)

### Custom tools

In [96]:
company_search_tool= create_company_search_tool(vector_store)
search_tool = create_all_companies_dense_search_tool(vector_store)
tools = [company_search_tool, search_tool]
llm_with_tools = llm.bind_tools(tools)
tools_by_name = {tool.name: tool for tool in tools}
query = "ik wil fysiotherapie vergelijken voor alle beschikbare verzekeringen, gebruik een beschikbare tool"

In [97]:
def answer_with_tool(query: str) -> List[BaseMessage]:
    messages: List[BaseMessage] = [HumanMessage(content=query)]
    ai_msg = llm_with_tools.invoke(messages)
    messages.append(ai_msg)

    # Check if it wants to use tools
    if ai_msg.tool_calls:

        # Execute the tools
        for tool_call in ai_msg.tool_calls:
            tool_result = tools_by_name[tool_call['name']].invoke(tool_call)
            messages.append(tool_result)

        # Ask again with tool results
        final_msg = llm_with_tools.invoke(messages)
        messages.append(final_msg)
        return messages  # This is the answer!
    else:
        # It answered directly without tools
        return messages
    
resultaten = answer_with_tool(query)

In [99]:
resultaten[3].content

'Ik heb de beschikbare dekkingsteksten doorzocht. Hieronder een samenvatting van de fysiotherapie-dekking per verzekeraar (bron: de gevonden dekkingsdocumenten).\n\nKort overzicht\n- Goudse (Expat Pakket Individueel)\n  - Fysiotherapie / oefentherapie (poliklinisch):  \n    - Standaard: niet gedekt  \n    - Optimaal: 20 behandelingen  \n    - Excellent: 30 behandelingen  \n  - Niet-klinische revalidatie: Optimaal 20 / Excellent 30 behandelingen (relevant als fysiotherapie als revalidatie wordt aangemerkt)  \n  - Klinische revalidatie (opname): Standaard 100% tot 60 dagen, Optimaal 100% tot 180 dagen, Excellent 100% tot 365 dagen\n\n- Henner (International Expat Insurance Package)\n  - Physiotherapy (outpatient):  \n    - Essential: niet gedekt  \n    - Bronze: 100% tot €1.000 per jaar  \n    - Gold: 100% tot €3.000 per jaar  \n  - Revalidatie na ziekenhuisopname: Gold 100% tot max. 28 dagen (Essential/Bronze niet gedekt)  \n  - Paramedische therapieën (bijv. ergotherapie/logopedie): Go

In [89]:
messages

[HumanMessage(content='Bij welke dekking bij goudse is fysiotherapie gedekt?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 380, 'total_tokens': 498, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CUBi1KGE5N0aFs6rcT8Gr0hOYeqgi', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--6f986d68-5323-4b90-a48c-d80b99931a95-0', tool_calls=[{'name': 'search_company_coverage', 'args': {'query': 'fysiotherapie vergoeding Goudse welke dekking fysiotherapie gedekt fysiotherapeut zorgverzekering aanvullend', 'company': 'Goudse', 'k': 10}, 'id': 'call_rYN3pIkKHdl

In [ ]:

tools_by_name = {tool.name: tool for tool in tools}

messages = [HumanMessage(content=query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    tool_result = tools_by_name[tool_call["name"]].invoke(tool_call)
    messages.append(tool_result)

final_response = llm_with_tools.invoke(messages)


In [72]:
for key, value in tool_messages.items():
    print(f"{key}: {value}/n")

messages: [ToolMessage(content="[(Document(metadata={'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-24T07:49:09.011550', 'filepath': '/app/data/documents/dekkingen/goudse_dekking.md', 'header_1': 'Expat Pakket Individueel - Premie- en dekkingsoverzicht 2025', 'header_2': '2. Ziektekosten', 'header_3': '2.1 Verzekerde rubrieken ziektekosten', 'header_4': 'Orthodontie', 'header_5': 'Overige orthodontie kinderen < 21 jaar', '_id': 'ce2d364f-f672-4364-a204-2f2f7d1c8c93', '_collection_name': 'dekkingen'}, page_content='##### Overige orthodontie kinderen < 21 jaar  \\nDit verwijst naar de vergoeding voor overige orthodontische zorg, zoals beugels, voor kinderen en jongeren tot 21 jaar, die niet gerelateerd is aan een schisis. Dit is niet gedekt onder het Standaard plan. Dit is niet gedekt onder het Optimaal plan. Voor het Excellent plan geldt een maximale vergoeding van € 1.000,- per verzekerde gedurende de gehele

In [ ]:
result = company_search_tool.invoke(ai_msg.tool_calls[0]['args'])

AttributeError: 'str' object has no attribute 'invoke'

In [46]:
ai_msg.tool_calls

[{'name': 'search_company_coverage',
  'args': {'query': 'beugelverzekering orthodontie beugel dekking',
   'company': 'Goudse',
   'k': 5},
  'id': 'call_9QYEhCaIAQQqXKLgznhf9Yxb',
  'type': 'tool_call'}]

In [51]:
for (doc, score) in result:
    print(doc.page_content)

##### Overige orthodontie kinderen < 21 jaar  
Dit verwijst naar de vergoeding voor overige orthodontische zorg, zoals beugels, voor kinderen en jongeren tot 21 jaar, die niet gerelateerd is aan een schisis. Dit is niet gedekt onder het Standaard plan. Dit is niet gedekt onder het Optimaal plan. Voor het Excellent plan geldt een maximale vergoeding van € 1.000,- per verzekerde gedurende de gehele verzekeringsduur.
#### Orthodontie  
##### Lip-, kaak-, verhemeltespleet  
Dit betreft de vergoeding voor orthodontische behandelingen die noodzakelijk zijn als gevolg van een aangeboren schisis (hazenlip) of een spleet in de kaak of het verhemelte. Dit is niet gedekt onder het Standaard plan. Dit is niet gedekt onder het Optimaal plan. Voor het Excellent plan is dit 100% gedekt.
##### Kaakchirurgie  
Dit omvat chirurgische ingrepen aan de kaak, vaak uitgevoerd door een kaakchirurg. Voor het Standaard plan is dit 100% gedekt. Voor het Optimaal plan is dit 100% gedekt. Voor het Excellent plan i

In [7]:
results = company_search_tool.invoke({"query": "ik zoek zwangerschaps dekking", "company": "Henner", "k": 5})

should=None min_should=None must=[FieldCondition(key='metadata.company', match=MatchValue(value='Henner'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None)] must_not=None


In [ ]:
for (doc, score) in results:
    print(doc.metadata)

{'document_name': 'henner_dekking.md', 'document_id': '79df16d23a93ed39', 'company': 'Henner', 'ingestion_timestamp': '2025-10-24T07:49:09.011839', 'filepath': '/app/data/documents/dekkingen/henner_dekking.md', 'header_1': 'International Expat Insurance Package - Benefit Overview 2025', 'header_2': 'Wijzigingen dekking pasgeborenen per 1-1-2025', '_id': '4c7c6a37-09a8-4659-99a1-6c6c7ef7b915', '_collection_name': 'dekkingen'}
{'document_name': 'henner_dekking.md', 'document_id': '79df16d23a93ed39', 'company': 'Henner', 'ingestion_timestamp': '2025-10-24T07:49:09.011839', 'filepath': '/app/data/documents/dekkingen/henner_dekking.md', 'header_1': 'International Expat Insurance Package - Benefit Overview 2025', 'header_2': 'International Medical and Assistance Insurance - Core Plan', 'header_4': 'PREGNANCY AND CHILDBIRTH (wachttijd 10 maanden)', 'header_5': 'Pregnancy (outpatient)', '_id': '33daf49e-4ecb-435b-a4ca-56e480169fc8', '_collection_name': 'dekkingen'}
{'document_name': 'henner_de

In [ ]:
query = "ik zoek zwangerschaps dekking"
dense_results = vector_store.similarity_search_with_score(
    query,
    k=5,
)

In [15]:
dense_results

[(Document(metadata={'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-24T07:49:09.011550', 'filepath': '/app/data/documents/dekkingen/goudse_dekking.md', 'header_1': 'Expat Pakket Individueel - Premie- en dekkingsoverzicht 2025', 'header_2': '2. Ziektekosten', 'header_3': '2.1 Verzekerde rubrieken ziektekosten', 'header_4': 'Poliklinisch', 'header_5': 'Dieetadvisering', '_id': 'f18657b0-74ea-499a-933d-30523546e867', '_collection_name': 'dekkingen'}, page_content='##### Dieetadvisering  \nDit verwijst naar de vergoeding voor advies en begeleiding door een diëtist. Dit is niet gedekt onder het Standaard plan. Voor het Optimaal plan is dit 100% gedekt. Voor het Excellent plan is dit 100% gedekt.'),
  0.5200777),
 (Document(metadata={'document_name': 'henner_dekking.md', 'document_id': '79df16d23a93ed39', 'company': 'Henner', 'ingestion_timestamp': '2025-10-24T07:49:09.011839', 'filepath': '/app/data/documents/dek

In [20]:
for (doc, score) in dense_results:
    print(doc.metadata)
    print(doc.page_content)
    print(f"Score: {score}\n")

{'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-24T07:49:09.011550', 'filepath': '/app/data/documents/dekkingen/goudse_dekking.md', 'header_1': 'Expat Pakket Individueel - Premie- en dekkingsoverzicht 2025', 'header_2': '2. Ziektekosten', 'header_3': '2.1 Verzekerde rubrieken ziektekosten', 'header_4': 'Poliklinisch', 'header_5': 'Dieetadvisering', '_id': 'f18657b0-74ea-499a-933d-30523546e867', '_collection_name': 'dekkingen'}
##### Dieetadvisering  
Dit verwijst naar de vergoeding voor advies en begeleiding door een diëtist. Dit is niet gedekt onder het Standaard plan. Voor het Optimaal plan is dit 100% gedekt. Voor het Excellent plan is dit 100% gedekt.
Score: 0.5200937

{'document_name': 'henner_dekking.md', 'document_id': '79df16d23a93ed39', 'company': 'Henner', 'ingestion_timestamp': '2025-10-24T07:49:09.011839', 'filepath': '/app/data/documents/dekkingen/henner_dekking.md', 'header_1': 'International E